# SOAP example

To install rascal:
(NOTE: See the top-level README for the most up-to-date installation instructions.)
+ mkdir ../build 
+ cd build
+ cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_TESTS=ON ..
+ make -j 4
+ make install

TODO: Compare against an existing SOAP implementation in Python (e.g. quippy)

In [1]:
%env OMP_NUM_THREADS=1

%matplotlib inline
from matplotlib import pyplot as plt

import os, sys
from ase.io import read
sys.path.insert(0,"../build/")

import sys
import time
import rascal
import json

import ase
#from ase.io import read
from ase.build import make_supercell
from ase.visualize import view
import numpy as np
import sys

from multiprocessing.dummy import Pool as ThreadPool
import queue
from copy import deepcopy,copy

import json

from rascal.neighbourlist import get_neighbourlist
from rascal.lib import RepresentationManager, FeatureManager
from rascal.representations.base import RepresentationFactory,FeatureFactory
from rascal.representations import SphericalExpansion
from rascal.neighbourlist.base import NeighbourListFactory
from rascal.utils import ostream_redirect

env: OMP_NUM_THREADS=1


In [2]:
frames = ase.io.read('../tests/reference_data/dft-smiles_500.xyz',index=':10')
new_frames = []
aa = []
for frame in frames:
    new_frames.append(make_supercell(frame,np.eye(3)*1))
print(np.max(list(map(len,new_frames))))

21


In [3]:
test_hypers = {"interaction_cutoff": 6.0, "cutoff_smooth_width": 1.0, "max_radial": 10, "max_angular": 8, "gaussian_sigma_type": "Constant", "gaussian_sigma_constant": 0.5}

In [4]:
sph_expn = rascal.representations.SphericalExpansion(**test_hypers)

In [5]:
with ostream_redirect():
    expansions = sph_expn.transform(new_frames)

In [6]:
X = expansions.get_feature_matrix().T

In [7]:
X.shape

(174, 3240)

In [8]:
%timeit -n 2 -r 1 expansions = sph_expn_test.transform(new_frames)

NameError: name 'sph_expn_test' is not defined

In [9]:
%timeit -n 2 -r 1 expansions = sph_expn.transform(new_frames)

29.2 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 2 loops each)


In [10]:
%timeit -n 2 -r 1 expansions = sph_expn_test.transform(new_frames)

NameError: name 'sph_expn_test' is not defined

In [11]:
expansions.get_feature_matrix()

array([[ 1.81265368e-02,  6.70588461e-04, -1.13732830e-04, ...,
         3.58312106e-02,  3.51795278e-02,  3.51978871e-02],
       [ 2.49537383e-02, -1.27385436e-04,  1.02888555e-04, ...,
         1.70805549e-02,  1.61612695e-02,  1.60618909e-02],
       [ 1.98733887e-02,  8.99783123e-03, -8.49293547e-05, ...,
         1.65957311e-02,  8.78261315e-03,  9.33905856e-03],
       ...,
       [-2.39384321e-03,  3.02827469e-03,  6.11525091e-04, ...,
        -4.38178598e-04, -3.57921530e-06, -3.13353104e-05],
       [-1.42037003e-03,  1.28100411e-03,  1.86352840e-04, ...,
        -1.77572050e-04, -2.05699126e-07, -1.28676148e-06],
       [-5.40155807e-04,  3.56175297e-04,  3.34704300e-05, ...,
        -4.46284092e-05, -2.98770807e-08, -4.89493122e-07]])

Good, should be 5 centres (5 atoms) * 10 radial * 81 ${}=(l+1)^2$